In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
import torch.nn as nn

In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
import nltk
from nltk.tokenize import sent_tokenize

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import textwrap

In [17]:
# 노트북 커널 프로세스에만 적용됩니다 (재부팅 필요 없음).
import os
os.environ["HF_HUB_DISABLE_XET"] = "1"

In [20]:
device = torch.device("cuda")
model_name = 'naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B'
model = AutoModelForCausalLM.from_pretrained(model_name)

In [21]:
model = model.half()
model = model.to(device)

In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [23]:
max_token_length = getattr(model.config, "max_position_embeddings", None) #getattr(tokenizer, "model_max_length", None)

#if max_token_length is None or max_token_length > 1e5:  # 무한대 설정인 경우
#            max_token_length = 4096   # 일반 LLM 기본값 가정

In [24]:
print(max_token_length)

131072


In [25]:
# 생성할 토큰 수 여유 확보
gen_new_tokens = 256
prompt_max = max(16, max_token_length - gen_new_tokens)

In [26]:
prompt_max

130816

In [64]:
def get_summarize(text, ksic='어떤 산업') :
    role = f'너는 뉴스를 요약하는 요약 전문가야. 다음 뉴스 기사를 {ksic}의 관점에서 요약해줘. 되도록 {ksic}과 관련된 핵심 단어를 포함해줘'
    #print(role)
    
    chat = [
              {"role": "tool_list", "content": ""},
              {"role": "system", "content": role}, 
              {"role": "user", "content": text},
    ]
    
    inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_dict=True, return_tensors="pt", 
                                          truncation=True, max_length=prompt_max)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    output_ids = model.generate(**inputs, max_length=max_token_length, stop_strings=["<|endofturn|>", "<|stop|>"], tokenizer=tokenizer)
    
    responses = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    
    _, _, answer = responses[0].partition("assistant\n")

    return answer.strip()

In [9]:
filename='summary_output.tsv'

In [10]:
df = pd.read_csv(filename, sep='\t')

In [11]:
len(df)

765026

In [12]:
df.loc[0]['text']

'1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등장해 화제다. 최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 1221m 초콜릿 파이를 만들었다. 1221m 초콜릿 파이는 크림 864kg, 우유 576리터, 밀가루 600kg, 버터 432kg, 초콜릿 360kg이 사용되었고 무게만도 4천207kg으로 3만 명이 먹을 수 있는 대단한 양이다. 1221m 초콜릿 파이를 만들기 위해 전문 제빵사 25명과 일반인 175명 등 총 200여명이 투입됐으며 3시간이 넘는 시간이 걸려 만든 것으로 알려졌다. 특히 1221m 초콜릿 파이는 ‘세계에서 가장 긴 밀푀유 파이’라는 명칭으로 기네스북에 등재되기도 했다. 종전까지 최고 기록을 가지고 있던 벨기에 파이(1037.25m)를 20년 만에 넘어섰다. 이를 접한 누리꾼들은 “1221m 초콜릿 파이 대단하다”, “1221m 초콜릿 파이 먹어보고 싶네”, “1221m 초콜릿 파이 다 먹었나?”, “1221m 초콜릿 파이 맛이 궁금해” 등의 반응을 보였다.'

In [28]:
df_best_scores = df[df['max_scores'] > 0.38]

In [59]:
df_best_scores[50:100]

,id,topic,text,ksic,max_scores,summary_new
50,NPRW2100000003.4624,S,"28번째 확진자, 3번째 환자와 지인 관계…'잠복기 14일 초과 논란' 경기도 고양...",c271,0.491929,"경기도 고양시에서 신종 코로나바이러스 감염증 28번째 환자가 발생한 가운데, 28번..."
51,NWRW1900000014.11964,S,전남도 “오늘은 미역 먹는 날” ‘5월 8일은 미역 먹는 날.’ 전남도해양수산과학원...,c102,0.490831,"전남도해양수산과학원이 어버이날, 미역국으로 효도하세요라는 캐치프레이즈를 내걸고 전남..."
52,NWRW1900000036.11620,S,천안 토종닭 사육 농가 AI 의심 신고 충남도는 21일 천안시 수신면 신풍리의 토종...,c101,0.490525,충남도는 21일 천안시 수신면 신풍리의 토종닭 사육 농가에서 조류인플루엔자(AI) ...
53,NWRW1900000045.5892,S,라면으로 만든 피자 잘라보니…“스프는 설마요” ‘짜파구리’에 이어 라면으로 만든 피...,c107,0.490400,16일 미국의 요리전문 사이트 슬라이스닷컴은 페페로니 라면 피자가 등장했다고 전하면...
54,NLRW2000000024.1667,S,홍성열 군수 “구제역 방역에 만전 기할 것” 증평군은 11일 군청 소회의실에서 구제...,c203,0.490238,증평군은 11일 군청 소회의실에서 구제역 방역비상태세유지 대책 회의를 개최하여 전국...
55,NWRW1900000044.8742,S,"자연원, 얼려먹는 유기농 블루베리·유기농 딸기 출시 건강전문기업 ㈜자연원이 유기농 ...",c103,0.490229,건강전문기업 (주)자연원이 유기농 100% 과일즙으로 만든 얼려먹는 유기농 블루베리...
56,NLRW1900000012.4093,S,15년 전과 달라진 게 없는 농가 평균소득 신음하는 농촌 회생을 위한 처방을 서둘러...,c103,0.490218,15년 전과 달라진 게 없는 농가 평균소득 신음하는 농촌 회생을 위한 처방을 서둘러...
57,NWRW1900000004.15085,S,"감자·양파도 제쳐… 파프리카, 채소 매출 1위 등극 파프리카 가 감자와 양파를 제치...",c103,0.489162,롯데마트는 5~6월 채소 매출을 분석한 결과 파프리카가 전체 채소 매출 가운데 35...
58,NLRW1900000078.3964,S,부실 농정이 부른 친환경농업 포기 속출 힘들게 고수해온 친환경 농업을 스스로 포기하...,c103,0.489149,국립농산물품질관리원에 따르면 친환경 농업 인증을 받았던 전남지역 2만1297농가가 ...
59,NWRW1900000004.6800,S,3월 말 햇상품 출하 앞두고 반값 된 양파 3~4월 햇상품 출하를 앞둔 양파와 마늘...,c103,0.489099,"3~4월 햇상품 출하를 앞둔 양파와 마늘 값이 하락하고 있는데, 지난 19일 서울 ..."


In [92]:
df_worst_scores = df[df['max_scores'] < 0.3]

In [114]:
df_worst_scores[100:150]

,id,topic,text,ksic,max_scores,summary_new
4336,NPRW1900000064.9022,S,"현대상선, 9년만에 ´고졸 공채´ 뽑는다 현대상선이 우수 고졸 인력을 공개 채용한다...",c311,0.298778,"현대상선이 2003년 이후 9년 만에 처음으로 우수 고졸 인력을 공개 채용하며, 입..."
4337,NPRW1900000040.7144,S,메르스 확진자 36%가 병원 방문객 중동호흡기증후군(메르스) 확진자 중 36%가 가...,c271,0.298774,"보건복지부 중앙메르스대책본부는 15일 메르스 감염자 분포가 병원 환자 47%, 가족..."
4338,NLRW2100000007.2425,S,코로나 신규 확진자 두 자릿수대 유지…완치자&gt;확진자 코로나19 확산세가 한풀 ...,c271,0.298767,18일 질병관리본부 중앙방역대책본부에 따르면 코로나19 확산세가 한풀 꺾이며 신규 ...
4339,NPRW2100000009.4805,S,73세 최고령 '25번 환자'…광둥성 방문했던 아들 부부도 확진 판정 경기 시흥에 ...,c271,0.298766,경기 시흥에 사는 73세 여성이 신종 코로나바이러스에 감염된 것으로 확인되었으며 같...
4340,NIRW2200000001.28577,S,"경주 7명 확진, 코로나19 사태 18개월 만에 ‘누적 800명’ 넘어 경북 경주에...",c271,0.298766,경북 경주에서 기존 확진자의 가족과 외국인 노동자를 중심으로 7명의 코로나19 확진...
4341,NWRW1900000035.17989,S,"정부, 진드기 바이러스 사망자 2명 더 확인 진드기 바이러스에 감염돼 사망한 것으로...",c271,0.298738,3일 질병관리본부는 중증열성혈소판감소증후군(SFTS) 의심사례 가운데 감염이 확인된...
4342,NPRW2000000004.47738,S,"담양군, 20-40 청년농업인 간담회 개최 전남 담양군은 최형식 군수 주재로 지난 ...",c103,0.298736,전남 담양군은 최형식 군수 주재로 지난 20일 담양군청 대회의실에서 청년 농업인들의...
4343,NWRW1900000012.8066,S,“훔친 치마 입으면 복” 슬쩍한 60대男 서울 중랑구의 주택에 사는 이모 씨(52·...,c141,0.298734,서울 중랑구의 주택에 사는 이모 씨(52·여)는 마당에 놓은 빨래 건조대에서 옷이 ...
4344,NWRW2200000004.1597,S,‘변이 신속검사’ 한달반 넘게 미적대다… 이제야 추진 방역 당국이 국내에 변이 코로...,c271,0.298728,"방역 당국이 국내에 변이 코로나 바이러스가 급속하게 확산하자, 길게는 일주일까지 걸..."
4345,NIRW2200000002.7493,S,부산 공동어시장 5명 추가 확진…새 감염원 부상 집단감염이 발생한 부산 서구 부산공...,c271,0.298726,집단감염이 발생한 부산 서구 부산공동어시장에서 지표환자인 3309번과 접촉한 5명이...


In [143]:
text = df_worst_scores.loc[4361]['text']
text

'바나나 전염병에 스미후루코리아 “국민과일 바나나의 매력!" 밝혀... 바나나 전염병에 스미후루코리아 “국민과일 바나나의 매력!" 밝혀... 바나나 전염병이 공개되면서 바나나가 멸종위기에 처해있다. 이러한 가운데 스미후루코리아 측은 “국민과일 바나나의 매력!”이라며 보도자료를 보냈다. 바나나 전염병에 맞서는 스미후루 코리아 측은 \'국민 과일\'은 무엇일까. 바나나다. 한 대형마트의 최근 조사에 따르면 바나나가 3년 연속 ‘가장 많이 팔리는 과일 1위’에 올랐다. 수박을 비롯해 감귤-사과-딸기 등 쟁쟁한 과일들을 앞선 결과다. 글로벌 청과 기업 \'스미후루코리아(www.sumifru.co.kr)\'는 \'국민과일\'로 등극한 바나나의 매력과 이유를 분석해 소개했다. 사시사철 저렴한 가격에 구입할 수 있다는 점은 바나나가 대중적으로 사랑 받는 장점. 이와 함께 바나나는 특유의 부드러운 과육과 매력적인 향으로 식욕을 돋군다. 이 때문에 아이들의 이유식으로, 성인 다이어트 식품으로 각광받고 있다. 바나나의 진정한 매력은 저렴한 가격뿐 아니라 풍부한 영양소에 있다. 또 다른 ‘국민과일’로 꼽히는 \'사과\'와 비교했을 때, 바나나의 영양이 얼마나 훌륭한 지 알 수 있다. 사과에는 100g 기준으로 칼로리가 75kcal 함유돼 있다. 반면, 바나나는 93kcal로 열량이 더 높다. 운동할 때 바나나를 섭취하면 많은 에너지를 빠르게 섭취할 수 있다. 바나나가 사과보다 칼로리가 높다고 해서 많이 섭취하면 다이어트에 방해가 되지 않을 지 걱정하는 경우가 있는데, 꼭 그렇지만은 않다. 바나나는 식이섬유가 다른 과일에 비해 풍부하다. 그래서 포만감을 쉽게 일으켜 폭식을 억제한다. 바나나에 다량 함유된 식이섬유는 고분자 탄수화물이다. 이 성분은 콜레스테롤 흡수를 막는다. 그 덕분에 성인병 예방에 탁월한 효과가 있으며, 위장의 공복감을 덜 느끼게 한다. 게다가 바나나에는 풍부한 칼륨(100g 당 358mg)이 함유돼 있다. 이는 사과(107mg)보다 3배 이상 높은 수치다. 바나나

In [144]:
get_summarize(text, '작물 재배업')

'바나나 전염병으로 인해 바나나가 멸종위기에 처해있음에도 불구하고, 스미후루코리아는 바나나의 매력을 강조하며 국민과일로서의 가치를 재확인했다. 스미후루코리아는 바나나가 사시사철 저렴한 가격에 제공되며, 부드러운 과육과 매력적인 향으로 인해 대중적인 사랑을 받고 있다고 설명했다. 또한, 바나나는 칼로리가 비교적 낮고 식이섬유가 풍부하여 다이어트와 건강에 이로운 식품으로 각광받고 있다. 특히, 칼륨 함량이 높아 체내 염분 배출과 혈압 조절에 효과적이라는 점이 강조되었다. 스미후루코리아는 전 세계에 걸쳐 유통하며 연간 5000만여 상자의 바나나를 공급하고 있다. 바나나 전염병에 대한 우려 속에서도 바나나는 여전히 많은 사람들에게 사랑받는 과일로 자리잡고 있다.'

In [145]:
get_summarize(text, '의약품 제조업')

'스미후루코리아는 최근 바나나 전염병이 대중의 관심을 끌고 있는 상황에서 "국민과일 바나나의 매력!"이라는 보도자료를 통해 바나나의 인기와 영양적 가치를 강조했습니다. 스미후루코리아는 바나나가 사시사철 저렴한 가격에 제공되며, 부드러운 과육과 매력적인 향으로 인해 많은 사람들에게 사랑받고 있다고 설명했습니다. 또한, 바나나는 칼로리가 비교적 낮고 식이섬유가 풍부하여 다이어트와 건강에 유익한 식품으로 각광받고 있습니다. 특히, 칼륨 함량이 높아 체내 염분 배출과 혈압 조절에 효과적이며, 성인병 예방에도 도움을 줍니다. 스미후루코리아는 이러한 바나나의 다양한 장점을 바탕으로 "국민과일"로 자리잡았다고 밝혔습니다. 한편, 바나나 전염병은 변종 파나마병 확산으로 인해 바나나가 멸종위기에 처했으나, 스미후루코리아는 이를 극복하기 위해 다양한 유통망을 통해 바나나를 지속적으로 공급하고 있습니다.'

In [14]:
df.to_pickle('summary_output.pkl')

In [40]:
get_summarize(df.loc[0]['text'])

'1221m 초콜릿 파이가 등장해 화제다. 최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 1221m 초콜릿 파이를 만들었다. 1221m 초콜릿 파이는 크림 864kg, 우유 576리터, 밀가루 600kg, 버터 432kg, 초콜릿 360kg이 사용되었고 무게만도 4천207kg으로 3만 명이 먹을 수 있는 대단한 양이다. 1221m 초콜릿 파이를 만들기 위해 전문 제빵사 25명과 일반인 175명 등 총 200여명이 투입됐으며 3시간이 넘는 시간이 걸려 만든 것으로 알려졌다. 특히 1221m 초콜릿 파이는 ‘세계에서 가장 긴 밀푀유 파이’라는 명칭으로 기네스북에 등재되기도 했다. 이를 접한 누리꾼들은 “1221m 초콜릿 파이 대단하다”, “1221m 초콜릿 파이 먹어보고 싶네”, “1221m 초콜릿 파이 다 먹었나?”, “1221m 초콜릿 파이 맛이 궁금해” 등의 반응을 보였다.'

In [62]:
df['summary_naver_llm'] = df['text'].progress_apply(get_summarize)

  0%|          | 0/765026 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [42]:
# text 열의 문자열 길이 계산
df["text_len"] = df["text"].str.len()

# 길이 기준으로 내림차순 정렬 후 상위 5개 추출
top1000 = df.nlargest(1000, "text_len")

# 결과 출력
print(top1000[["text", "text_len"]])

                                                     text  text_len
697187  [소재부품 국산화, 다시 시작하자] <10>결산 좌담회 본지는 7월 1일 일본 정부...     10843
689300  [CES 2019 결산 좌담회]"4차 산업혁명시대, 오픈이노베이션으로 대응하자" 미...     10549
546824  돼지열병 창궐하자 “대박…” 비극을 기다린 사람들 비극도 돈이 되는 시대는 치사율 ...     10232
698436  [2019 신년기획]결산 좌담회 '4차 산업혁명 해법을 찾자' 대한민국을 휘감는 대...      9657
698445  ‘집단지성의 성채’ 위키백과, ‘지식정보의 편향’ 넘을 수 있을까 ‘개방과 공유’....      8992
...                                                   ...       ...
694242  끈질긴 '구청장 친인척' 보도, 지역 채널의 존재 이유 정부가 LG유플러스의 CJ헬...      4181
79775   발사 임박했다는 북한 미사일, 그 실체는? 북한이 미사일 발사 준비를 완료한 것으로...      4180
650577  [아주초대석] 김희관 더블미 대표 “현실로 나온 판타지 세상··· 할머니도 만들 수...      4180
698398  공인인증서만? 30년 낡은 '요금인가제도' 사라진다 오랜 기간 낡은 규제로 지적돼 ...      4180
761287  개인정보 안 주면 지도 업데이트 안돼~ '아이나비' 브랜드로 업계 1위를 차지하고 ...      4180

[1000 rows x 2 columns]


In [72]:
top1000['summary_naver_llm'] = top1000['text'].progress_apply(get_summarize)

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [44]:
get_summarize(top5.iloc[0]['text'])

'이 기사에서는 소재부품 국산화 현황과 문제점, 대안에 대해 논의하는 가운데, 특히 반도체, 디스플레이, 배터리 등 다양한 산업 분야에서 국산화의 필요성과 전략에 대해 다루고 있습니다. 일본 정부의 수출규제 이후 국산화 관심가 높아졌으며, 전문가들은 국산화의 필요성을 강조하고 있습니다. 정부는 소재부품 국산화를 위해 R&D 전략과 글로벌 협력 등을 강조하며, 중소기업과 대기업 간의 협력도 중요하다고 지적합니다. 또한, 기초과학 연구와 정부의 소재산업 육성 정책의 중요성을 언급하며, 장기적인 관점에서 국산화를 추진해야 한다고 강조합니다.'

In [47]:
top5['summary_naver_llm'] = top5['text'].progress_apply(get_summarize)

  0%|          | 0/5 [00:00<?, ?it/s]

In [48]:
top5

,id,topic,text,ksic,max_scores,summary_new,text_len,summary_naver_llm
697187,NPRW2000000005.14858,T,"[소재부품 국산화, 다시 시작하자] <10>결산 좌담회 본지는 7월 1일 일본 정부...",c303,0.015210,일본 조치 이후 약 한 달간 소재부품 국산화와 공급망 다변화에 대한 관심이 커지면서...,10843,"이 기사에서는 소재부품 국산화 현황과 문제점, 대안에 대해 논의하는 가운데, 특히 ..."
689300,NPRW2000000005.14771,T,"[CES 2019 결산 좌담회]""4차 산업혁명시대, 오픈이노베이션으로 대응하자"" 미...",c262,0.020692,미국 라스베이거스에서 8일(현지시간)부터 11일까지 열렸던 'CES 2019'는 한...,10549,'CES 2019'에서 논의된 오픈이노베이션은 기업에 필요한 기술과 자원을 외부에서...
546824,NWRW2000000002.25857,T,돼지열병 창궐하자 “대박…” 비극을 기다린 사람들 비극도 돈이 되는 시대는 치사율 ...,c101,0.070129,연천군 백학면의 양돈농장에서 아프리카돼지열병(ASF·이하 돼지열병)이 확진됐다. 전...,10232,아프리카돼지열병(AST)이 파주를 덮치면서 돼지열병을 테마로 삼은 투자자들과 기업들...
698436,NPRW2000000005.14769,T,[2019 신년기획]결산 좌담회 '4차 산업혁명 해법을 찾자' 대한민국을 휘감는 대...,c211,0.009373,4차 산업혁명은 제조업을 기반으로 고도 압축 성장을 거듭한 우리나라에 새로운 성장엔...,9657,2019 신년기획으로 열린 '4차 산업혁명 해법' 좌담회에서 전문가들은 대한민국이 ...
698445,NWRW2000000002.25495,T,"‘집단지성의 성채’ 위키백과, ‘지식정보의 편향’ 넘을 수 있을까 ‘개방과 공유’....",c272,0.008868,"최근 국내 통계물리학자들이 방대한 위키 빅데이터를 분석해, 적은 비율의 열성적 편집...",8992,"위키백과는 2001년 1월 15일 설립된 이후, 전 세계적으로 수많은 사용자와 편집..."


In [77]:
df.columns

Index(['id', 'topic', 'text', 'ksic', 'max_scores'], dtype='object')

In [78]:
df.iloc[0].text

'코로나 환자 늘어 일반 중환자 치료 공백 코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코로나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다. 의료 현장에서는 “코로나 확산으로 암 환자 등 일반 중환자 치료에 공백이 생기고 있다”는 목소리가 나온다. 18일 김범석 서울대 혈액종양내과 교수팀이 2019~2020년 서울대병원에서 사망한 암 환자 1456명을 분석한 결과 2019년 서울대병원 응급실에서 세상을 떠난 암 환자는 53명에서 2020년 99명으로 약 86% 대폭 증가한 것으로 나타났다. 말기 암 환자의 응급실 체류 시간도 늘었고, 환자의 생존을 위해 강제로 혈압을 높이는 승압제를 사용한 환자 수도 52.3%에서 59.2%로 약 7%포인트 늘었다. 심폐소생술을 받은 암 환자도 12.5%에서 16.3%로 증가했다. 김 교수는 “코로나 환자도 중요하지만, 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다”고 말했다.'

In [26]:
run_summarize(df.iloc[0].text, tokenizer, model, device)

============ original ===============
코로나 환자 늘어 일반 중환자 치료 공백 코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코로나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다 의료 현장에서는 코로나 확산으로 암 환자 등 일반 중환자 치료에 공백이 생기고 있다는 목소리가 나온다 18일 김범석 서울대 혈액종양내과 교수팀이 20192020년 서울대병원에서 사망한 암 환자 1456명을 분석한 결과 2019년 서울대병원 응급실에서 세상을 떠난 암 환자는 53명에서 2020년 99명으로 약 86 대폭 증가한 것으로 나타났다 말기 암 환자의 응급실 체류 시간도 늘었고 환자의 생존을 위해 강제로 혈압을 높이는 승압제를 사용한 환자 수도 523에서 592로 약 7포인트 늘었다 심폐소생술을 받은 암 환자도 125에서 163로 증가했다 김 교수는 코로나 환자도 중요하지만 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다고 말했다


/home/osung/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


464 283
time to summarize is 2.20798 sec
============ summary  ================
코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코코나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다 의료 현장에서는 코로나가 확산으로 암 환자 등 일반 중 환자 치료에 공백이 생기고 있다는 목소리가 나온다. 김 교수는 코로나도 중요하지만 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다고 말했다. 코로나의 확산으로 일반 중환자의 치료가 공백이 생긴다는 목소리가 나오고 있다. 김교수는 코로나는 물론 이 때문에 방치된 환자들이 많다는 사실을 알아야 한다 말했다.
464 283
=============== long summary  ================
코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코코나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다 의료 현장에서는 코로나가 확산으로 암 환자 등 일반 중 환자 치료에 공백이 생기고 있다는 목소리가 나온다. 김 교수는 코로나도 중요하지만 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다고 말했다. 코로나의 확산으로 일반 중환자의 치료가 공백이 생긴다는 목소리가 나오고 있다. 김교수는 코로나는 물론 이 때문에 방치된 환자들이 많다는 사실을 알아야 한다 말했다.
time to summarize long sentence is 1.32148 sec


In [28]:
run_summarize(df.iloc[1].text, tokenizer, model, device)

============ original ===============
 휴가때 골프 꼭 치고 싶으면 자비로 쳐라 허태열 대통령비서실장이 최근 수석비서관회의를 주재하면서 휴가 때 골프를 꼭 치고 싶은 사람은 문제가 되지 않을 사람과 자비로 쳐도 된다며 웬만하면 필드골프장 대신 스크린골프를 이용하는 게 좋겠다고 말한 것으로 전해졌다 2월 박근혜 정부 출범 이후 골프 금지령이 내려진 적은 없지만 북한의 위협이 이어진 데다 정권 출범의 긴장감이 더해져 공직자들은 자연스레 골프를 칠 엄두를 내지 못했다 이와 관련해 박 대통령은 10일 언론사 논설실장 오찬에서 캐디들도 수입이 그렇고 자꾸 외국만 나가서 어떻게 하느냐고 걱정하는 이야기도 있다 지금 여러 가지 생각을 하고 있다고 밝혔다 청와대 관계자는 자비를 들여 취미로 치는 것까지 문제 삼을 수야 있겠느냐며 그렇다고 이제부터 골프를 쳐도 된다고 선언하는 것도 우습고 자연스럽게 분위기가 잡힐 것이라고 말했다 허 실장의 발언이 알려지자 관가에선 골프를 쳐도 된다는 건지 치지 말라는 건지 헷갈린다는 반응이 나왔다
497 317
time to summarize is 1.45685 sec
============ summary  ================
허태열 대통령비서실장이 최근 수석비서관회의를 주재하면서 휴가 때 골프를 꼭 치고 싶은 사람은 문제가 되지 않을 사람과 자비로 쳐도 된다며 웬만하면 필드골프장 대신 스크린골프를 이용하는 게 좋겠다고 말한 것으로 전해졌다 2월 박근혜 정부 출범 이후 골프 금지령이 내려진 적은 없지만 북한의 위협이 이어진 데다 정권 출범의 긴장감이 더해져 공직자들은 자연스레 골프를 칠 엄두를 내지 못했다 이와 관련해 박 대통령은 10일 언론사 논설실장 오찬에서 캐디들도 수입이 그렇고 자꾸 외국만 나가서 어떻게 하느냐고 걱정하는 이야기도 있다 지금 여러 가지 생각을 하고 있다고 밝혔다
497 317
=============== long summary  ================
허태열 대통령비서실장

In [30]:
df.columns

Index(['Unnamed: 0', 'id', 'topic', 'text', 'ksic', 'max_scores', 'keywords',
       'keywords_orig', 'ksic_scores', 'max_ksic', 'ksic_count',
       'max_ksic_scores'],
      dtype='object')

In [31]:
df.iloc[0]

Unnamed: 0                                                         0
id                                              NWRW2200000004.11879
topic                                                              P
text               코로나 환자 늘어 일반 중환자 치료 공백 코로나 바이러스 확산으로 위중증 환자가 급...
ksic                                                            c271
max_scores                                                    0.4621
keywords           {'김': 1, '분석': 1, '바이러스': 1, '코로나': 5, '병원': 2...
keywords_orig      {'환자': 11, '코로나': 5, '암': 5, '일반': 3, '중환자': 3...
ksic_scores        {'c102': 1, 'c272': 1, 'c211': 2, 'c271': 24, ...
max_ksic                                                    ['c271']
ksic_count                                                         1
max_ksic_scores                                                   24
Name: 0, dtype: object

In [66]:
df = df.drop(columns='Unnamed: 0')

In [33]:
df2.columns

Index(['id', 'topic', 'text', 'ksic', 'max_scores', 'keywords',
       'keywords_orig', 'ksic_scores', 'max_ksic', 'ksic_count',
       'max_ksic_scores'],
      dtype='object')

In [34]:
df = df2

In [79]:
df_sorted = df.sort_values(by='text', key=lambda x: x.str.len(), ascending=False)

In [81]:
df_sorted.iloc[0].text

"[전문] 문재인 정부 2년 특집 대담, 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 다음은 일문일답. ◆취임 2주년 소회 -2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다. ▷먼저 국민들께 감사인사부터 드려야겠다. 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고, 저를 대통령으로 선택해 주셨다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 얼마나 기대에 부응했는지 잘 모르겠다. 많은 성과들이 있었다고 생각한다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다. ◆국내 정치 -국내 정치 현안 문제로 들어가야 할 것 같다. 조금 전에 대북 현안들, 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나. ▷이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다. 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다. -제가 여쭤본 것은 국내 정치권의…. ▷그렇다. 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만, 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다. 처리해야 할 민생법안도 많이 있고, 추경 문제도 논의해야 되고, 그래서 이런 국면에서 필요한 것이 지난번에 합의했던 여야정 

In [86]:
len(df_sorted.iloc[0].text)

24234

In [90]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=1024, verify=False)

============ original ===============
[전문] 문재인 정부 2년 특집 대담, 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 다음은 일문일답. ◆취임 2주년 소회 -2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다. ▷먼저 국민들께 감사인사부터 드려야겠다. 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고, 저를 대통령으로 선택해 주셨다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 얼마나 기대에 부응했는지 잘 모르겠다. 많은 성과들이 있었다고 생각한다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다. ◆국내 정치 -국내 정치 현안 문제로 들어가야 할 것 같다. 조금 전에 대북 현안들, 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나. ▷이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다. 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다. -제가 여쭤본 것은 국내 정치권의…. ▷그렇다. 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만, 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다. 처리해야 할 민생법안도 많이 있고, 추경 문제도 논의해야 

24234 446
time to summarize is 5.79661 sec

============  summary  ================
문재인 대통령은 남북 정상회담에서 북한의 도발에 대해 대화와 협상을 통해 해결책을 찾겠다고 말했다. 대통령은 남북 정상회담이 성공적으로 개최될 수 있도록 최선을 다해야 한다고 말했다. 또한 남북이 대화의 장을 만들 수 있도록 노력해야 한다고 했다. 또한, 남북의 대화에 대한 대화의 장이 될 수 있도록 해야 한다. 또한 우리 정부는 남북의 평화적 협력의 길을 열어 나가야 한다고 생각한다. 또한 북한의 비핵화를 위해 노력해야 할 것이다. 또한 우리는 남북이 평화적으로 협력할 수 있는 길을 만들어 나가야 한다. -.,. ; 의 을 통해 남북의 길을 열어야 한다. 우리는 이 아니라 로 가는 길이다.의 길로 가야 한다.  남북의 길을 해 주길 바란다. 는데 라고 생각한다. 우리는 남북의 길로 나아가야 한다. 남북의 길의 길을 더 나아가야 한다고 생각한다며, 남북관계를 개선해야 한다. 우리 정부는 
1002 344
991 321
972 314
991 317
973 283
882 331
919 307
986 321
1012 292
972 301
996 323
1010 332
1006 321
1006 319
926 312
998 448
957 373
997 280
961 315
986 306
958 288
1012 290
964 312
971 436
342 262

===============  long summary  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길

902 281
936 307
936 351
928 366
1014 342
1017 315
1008 398
940 313
279 360

===============  long summary2  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다.제가 분명하게 검찰에 말하고 싶은 것은 공수처도 수사권 조정법안도 그렇고, 지금까지 검찰이 사정기구로 본연의 역할을 다하지 못했기 때문에 개혁의 방안으로 논의가 되는 것이다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다. 민정수석이 가장 중요한 정부에서의 책임들이 인사검증과 권력기관들의 개혁이 제일 중요한 임무 가운데 하나다. 청와대 검증부터 청문회까지 전체가 하나의 검증 과정인 것이다.인사청문회가 정쟁쟁의 장으로 운영된다면 좋은 인사를 발탁하는 장이 아니라 좋은 인사를 막는 장이 될 것이다. 총리, 장관들이 정치에 나선다면 그것은 전적으로 본인 의사에 달려 있는 것이고, 대통령으로서 바라고 싶은 것은 선거에 나갈 생각이 있다면 선거시기에 임박해서가 아니라 충분한 여유를 두고 의사를 밝히는 것이 바람직하겠다고 생각하고 있다. 또한 요즘 유권자들의 요구도, 이제는 낙하산으로 공천 받아 내려오는 게 아니라 지역에서 지역주민과 밀착되길 바라기 때문에 그것이 유권자들 요구라고도 생각한다. 그뿐만 아

In [92]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=2048, verify=False)

============ original ===============
[전문] 문재인 정부 2년 특집 대담, 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 다음은 일문일답. ◆취임 2주년 소회 -2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다. ▷먼저 국민들께 감사인사부터 드려야겠다. 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고, 저를 대통령으로 선택해 주셨다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 얼마나 기대에 부응했는지 잘 모르겠다. 많은 성과들이 있었다고 생각한다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다. ◆국내 정치 -국내 정치 현안 문제로 들어가야 할 것 같다. 조금 전에 대북 현안들, 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나. ▷이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다. 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다. -제가 여쭤본 것은 국내 정치권의…. ▷그렇다. 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만, 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다. 처리해야 할 민생법안도 많이 있고, 추경 문제도 논의해야 

24234 446
time to summarize is 6.46697 sec

============  summary  ================
문재인 대통령은 남북 정상회담에서 북한의 도발에 대해 대화와 협상을 통해 해결책을 찾겠다고 말했다. 대통령은 남북 정상회담이 성공적으로 개최될 수 있도록 최선을 다해야 한다고 말했다. 또한 남북이 대화의 장을 만들 수 있도록 노력해야 한다고 했다. 또한, 남북의 대화에 대한 대화의 장이 될 수 있도록 해야 한다. 또한 우리 정부는 남북의 평화적 협력의 길을 열어 나가야 한다고 생각한다. 또한 북한의 비핵화를 위해 노력해야 할 것이다. 또한 우리는 남북이 평화적으로 협력할 수 있는 길을 만들어 나가야 한다. -.,. ; 의 을 통해 남북의 길을 열어야 한다. 우리는 이 아니라 로 가는 길이다.의 길로 가야 한다.  남북의 길을 해 주길 바란다. 는데 라고 생각한다. 우리는 남북의 길로 나아가야 한다. 남북의 길의 길을 더 나아가야 한다고 생각한다며, 남북관계를 개선해야 한다. 우리 정부는 
1993 285
2026 277
2024 244
2019 366
1994 268
1985 324
2020 343
2043 350
2024 327
2030 351
2034 353
1598 411

===============  long summary  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.검찰 스스로 개혁을 할 수 있는 많은 기회들을 지

In [94]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=1024, verify=False)

============ original ===============
[전문] 문재인 정부 2년 특집 대담, 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 다음은 일문일답. ◆취임 2주년 소회 -2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다. ▷먼저 국민들께 감사인사부터 드려야겠다. 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고, 저를 대통령으로 선택해 주셨다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 얼마나 기대에 부응했는지 잘 모르겠다. 많은 성과들이 있었다고 생각한다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다. ◆국내 정치 -국내 정치 현안 문제로 들어가야 할 것 같다. 조금 전에 대북 현안들, 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나. ▷이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다. 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다. -제가 여쭤본 것은 국내 정치권의…. ▷그렇다. 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만, 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다. 처리해야 할 민생법안도 많이 있고, 추경 문제도 논의해야 

24234 446
time to summarize is 6.44101 sec

============  summary  ================
문재인 대통령은 남북 정상회담에서 북한의 도발에 대해 대화와 협상을 통해 해결책을 찾겠다고 말했다. 대통령은 남북 정상회담이 성공적으로 개최될 수 있도록 최선을 다해야 한다고 말했다. 또한 남북이 대화의 장을 만들 수 있도록 노력해야 한다고 했다. 또한, 남북의 대화에 대한 대화의 장이 될 수 있도록 해야 한다. 또한 우리 정부는 남북의 평화적 협력의 길을 열어 나가야 한다고 생각한다. 또한 북한의 비핵화를 위해 노력해야 할 것이다. 또한 우리는 남북이 평화적으로 협력할 수 있는 길을 만들어 나가야 한다. -.,. ; 의 을 통해 남북의 길을 열어야 한다. 우리는 이 아니라 로 가는 길이다.의 길로 가야 한다.  남북의 길을 해 주길 바란다. 는데 라고 생각한다. 우리는 남북의 길로 나아가야 한다. 남북의 길의 길을 더 나아가야 한다고 생각한다며, 남북관계를 개선해야 한다. 우리 정부는 
1002 344
991 321
972 314
991 317
973 283
882 331
919 307
986 321
1012 292
972 301
996 323
1010 332
1006 321
1006 319
926 312
998 448
957 373
997 280
961 315
986 306
958 288
1012 290
964 312
971 436
342 262

===============  long summary  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길

902 281
936 307
936 351
928 366
1014 342
1017 315
1008 398
940 313
279 360

===============  long summary2  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다.제가 분명하게 검찰에 말하고 싶은 것은 공수처도 수사권 조정법안도 그렇고, 지금까지 검찰이 사정기구로 본연의 역할을 다하지 못했기 때문에 개혁의 방안으로 논의가 되는 것이다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다. 민정수석이 가장 중요한 정부에서의 책임들이 인사검증과 권력기관들의 개혁이 제일 중요한 임무 가운데 하나다. 청와대 검증부터 청문회까지 전체가 하나의 검증 과정인 것이다.인사청문회가 정쟁쟁의 장으로 운영된다면 좋은 인사를 발탁하는 장이 아니라 좋은 인사를 막는 장이 될 것이다. 총리, 장관들이 정치에 나선다면 그것은 전적으로 본인 의사에 달려 있는 것이고, 대통령으로서 바라고 싶은 것은 선거에 나갈 생각이 있다면 선거시기에 임박해서가 아니라 충분한 여유를 두고 의사를 밝히는 것이 바람직하겠다고 생각하고 있다. 또한 요즘 유권자들의 요구도, 이제는 낙하산으로 공천 받아 내려오는 게 아니라 지역에서 지역주민과 밀착되길 바라기 때문에 그것이 유권자들 요구라고도 생각한다. 그뿐만 아

In [96]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=2048, verify=False)

24234 446
1993 285
2026 277
2024 244
2019 366
1994 268
1985 324
2020 343
2043 350
2024 327
2030 351
2034 353
1598 411
1928 296
1928 345


In [59]:
ss = seperate_sentence(text, 1024)

In [60]:
ss

['',
 '전문 문재인 정부 2년 특집 대담 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS 문재인 정부 2년 특집 대담 대통령에게 묻는다에 출연해 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다 다음은 일문일답 취임 2주년 소회 2년 전 오늘 이 시간 기억하실지 모르겠다 나라다운 나라를 만들어 달라 이런 요구 속에서 문재인 정권이 출범을 했다 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다 먼저 국민들께 감사인사부터 드려야겠다 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고 저를 대통령으로 선택해 주셨다 그래서 문재인 정부는 촛불정신 위에 서 있다 촛불민심이 명하는 대로 국정농단 그리고 반칙과 특권이라는 적폐시대를 마감하고 새로운 시대 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다 얼마나 기대에 부응했는지 잘 모르겠다 많은 성과들이 있었다고 생각한다 그러나 한편으로 또 아쉬운 부분들도 많이 있고 또 보완해야 할 과제들도 많이 있다고 느낀다 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다 국내 정치 국내 정치 현안 문제로 들어가야 할 것 같다 조금 전에 대북 현안들 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나 이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다 제가 여쭤본 것은 국내 정치권의 그렇다 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다 처리해야 할 민생법안도 많이 있고 추경 문제도 논의해야 되고 그래서 이런 국면에서 필요한 것이 지난번에 합의했던 여야정 상설국정협의체를 가동하는 것이라고 생각한다 우선적으로 방금 현안으로 대두된

In [61]:
len(ss)

2

In [100]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=2048, verify=False)

1993 285
2026 277
2024 244
2019 366
1994 268
1985 324
2020 343
2043 350
2024 327
2030 351
2034 353
1598 411
1928 296
1928 345


"문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.한일 관계문제다. 과거사 문제는 과거를 지울 수 없는 문제고 과거사 문제가 한일관계의 족쇄가 된 게 너무 오랜 시간이다. 다만, 어려움을 겪고 있는 것은 과거사 문제로 한번씩 양국관계발전에 발목을 잡고 있는데 그것은 결코 한국정부가 만들어 내고 있는 문제가 아니다. 하지만 일본 정치지도자들이 자꾸 그 문제를 국내 정치적인 문제로 다루기 때문에 미래지향적인 발전의 발목을 잡는 일이 거듭되고 있다고 본다. -제일 문제는 청년일자리다. 제조업의 혁신해 고도화해서 경쟁력을 높여 일자리를 늘리는 방향이 하나 있고, 신산업들을 빨리 성장시켜서 좋은 일자리를 만들어야겠고, 요즘 벤처창업들을 크게 늘리고 지원할 필요가 있다."

In [101]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=1024, verify=False)

1002 344
991 321
972 314
991 317
973 283
882 331
919 307
986 321
1012 292
972 301
996 323
1010 332
1006 321
1006 319
926 312
998 448
957 373
997 280
961 315
986 306
958 288
1012 290
964 312
971 436
342 262
902 281
936 307
936 351
928 366
1014 342
1017 315
1008 398
940 313
279 360
997 396
991 338
1018 317
1043 311


"문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다."

In [102]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=1536, verify=False)

1517 285
1448 304
1510 269
1505 232
1466 377
1489 361
1399 331
1477 383
1419 354
1414 402
1509 315
1497 362
1493 339
1488 334
1526 283
1633 411
1395 416
1384 392
1358 328
1153 333


"문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 그 해법을 선택한 것을 가지고 독재라 하는 것은 정말 조금 맞지 않는 얘기란 말씀을 드린다. 검찰 스스로 개혁을 할 수 있는 많은 기회들을 지금까지 놓쳐 왔다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.북한이 며칠 전에 여러 종류의 단거리 발사체를 발사한 데 이어서 오늘은 일단 단거리 미사일로 추정되는 그런 발사를 했다. 유엔 안보리 결의 속에는 탄도미사일을 하지 말라는 그런 표현이 있기 때문에 비록 단거리라 할지라도 그것이 탄도미사일일 경우에는 유엔안보리 결의에 위반될 소지도 없지 않다고 생각한다. 그런 방식으로 북한의 의도를 여러 가지로 해석하게 만들고 또 우려하게 만들고 자칫 잘못하면 대화와 협상국면의 찻물을 끼얹는 이런 성격을 거듭 하는 것은 결코 바람직하지 않다는 것을 북한측에 다시 한 번 이야기하고 싶다. 그런 형편이기 때문에 북한동포들의 심각한 기아상태를 우리가 외면할 수 없고, 우리가 동포애나 인도주의적인 차원에서라도 우리가 북한의 식량을 지원할 필요가 있다고 생각하는 것이다.북한의 식량지원 문제에 대해서 저로서는 가장 바람직한 것은 지금 패스트트랙 문제 때문에 여야 간 정국이 교착상태에 빠져 있는데 그 문제는 별도로 해결하더라도 북한에 대한 식량지원에 대해서는 대통령과 여야가 함께 모여서 협의를 하는 것이 바람직하지 않을까 라는 생각을 하고 있다. 그래서 그렇기 때문에 사실은 식량지원의 문제에 대해서 우리가 한

In [103]:
t = "문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다."

In [105]:
len(t)

311

In [106]:
len("문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.한일 관계문제다. 과거사 문제는 과거를 지울 수 없는 문제고 과거사 문제가 한일관계의 족쇄가 된 게 너무 오랜 시간이다. 다만, 어려움을 겪고 있는 것은 과거사 문제로 한번씩 양국관계발전에 발목을 잡고 있는데 그것은 결코 한국정부가 만들어 내고 있는 문제가 아니다. 하지만 일본 정치지도자들이 자꾸 그 문제를 국내 정치적인 문제로 다루기 때문에 미래지향적인 발전의 발목을 잡는 일이 거듭되고 있다고 본다. -제일 문제는 청년일자리다. 제조업의 혁신해 고도화해서 경쟁력을 높여 일자리를 늘리는 방향이 하나 있고, 신산업들을 빨리 성장시켜서 좋은 일자리를 만들어야겠고, 요즘 벤처창업들을 크게 늘리고 지원할 필요가 있다.")

641

In [107]:
len("문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 그 해법을 선택한 것을 가지고 독재라 하는 것은 정말 조금 맞지 않는 얘기란 말씀을 드린다. 검찰 스스로 개혁을 할 수 있는 많은 기회들을 지금까지 놓쳐 왔다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.북한이 며칠 전에 여러 종류의 단거리 발사체를 발사한 데 이어서 오늘은 일단 단거리 미사일로 추정되는 그런 발사를 했다. 유엔 안보리 결의 속에는 탄도미사일을 하지 말라는 그런 표현이 있기 때문에 비록 단거리라 할지라도 그것이 탄도미사일일 경우에는 유엔안보리 결의에 위반될 소지도 없지 않다고 생각한다. 그런 방식으로 북한의 의도를 여러 가지로 해석하게 만들고 또 우려하게 만들고 자칫 잘못하면 대화와 협상국면의 찻물을 끼얹는 이런 성격을 거듭 하는 것은 결코 바람직하지 않다는 것을 북한측에 다시 한 번 이야기하고 싶다. 그런 형편이기 때문에 북한동포들의 심각한 기아상태를 우리가 외면할 수 없고, 우리가 동포애나 인도주의적인 차원에서라도 우리가 북한의 식량을 지원할 필요가 있다고 생각하는 것이다.북한의 식량지원 문제에 대해서 저로서는 가장 바람직한 것은 지금 패스트트랙 문제 때문에 여야 간 정국이 교착상태에 빠져 있는데 그 문제는 별도로 해결하더라도 북한에 대한 식량지원에 대해서는 대통령과 여야가 함께 모여서 협의를 하는 것이 바람직하지 않을까 라는 생각을 하고 있다. 그래서 그렇기 때문에 사실은 식량지원의 문제에 대해서 우리가 한미간에 합의를 한 것이 이번 발사 이전인데, 그 이후에 또 다시 발사가 있었기 때문에 이 점에 대해서는 국민들의 공감이나 제재가 필요하다고 생각한다. 한일관계는 굉장히 중요하다고 생각하고, 앞으로 더 미래지향적으로 발전돼 나가야 한다고 생각한다.고령화가 급격하게 진행되고 있어서 65세 이상 인구가 14%가 넘는 고령사회를 이미 2017년에 통과했고, 2025년이면 20%가 넘는 초고령사회가 될 것으로 예상된다. 고령인구가 크게 늘고 있기 때문에 일자리 수를 더 늘리고 과거의 급여가 낮았기 때문에 급여를 두배 높여서 실제로 어르신들의 빈곤 해결에 도움되도록 정부가 노력하고 있는 것이다. 거시적인 경제성공은 인정하고 자부심을 가져야 한다고 생각한다. 다만 국민들에게 고르게 다 소득배분이 되고 있지 않고 있기 때문에 아직도 양극화가 심각하고 특히 소득이 낮은 층의 소득이 늘지 않고 있기때문에 그분들의 사는 문제가 해결이 안 되고 있다.")

1469

In [109]:
filename2='/home/osung/data/korean/modu/json/combined_news.tsv'

In [115]:
df2 = pd.read_csv(filename2, sep='\t')

In [116]:
len(df2)

3611424

In [112]:
df2.columns

Index(['id', 'topic', 'text', 'ksic', 'max_scores'], dtype='object')

In [113]:
len(df)

1530052

In [114]:
df.columns

Index(['id', 'topic', 'text', 'ksic', 'max_scores'], dtype='object')